<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/05_combined_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-07-28 08:19:01--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11848 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.57K  --.-KB/s    in 0s      

2022-07-28 08:19:01 (83.7 MB/s) - ‘utils.py’ saved [11848/11848]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 78.8 MB/s 


In [4]:
BATCH_SIZE = 16
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'data')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 20092 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
hsv_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)

rgb_features = rgb_hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = xyz_hub_module(hsv_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features, hsv_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [10]:
model._name = 'combined-model-v1'

In [11]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [12]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy', 
                                              verbose=1,
                                              patience=5,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name)
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [13]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15
1256/1256 [==============================] - ETA: 0s - loss: 13.5640 - auc: 0.8150 - accuracy: 0.7332

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


1256/1256 [==============================] - 2047s 2s/step - loss: 13.5640 - auc: 0.8150 - accuracy: 0.7332 - val_loss: 0.8218 - val_auc: 0.8285 - val_accuracy: 0.7539
Epoch 2/15
1256/1256 [==============================] - ETA: 0s - loss: 7.8931 - auc: 0.9163 - accuracy: 0.8380

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


...

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


1256/1256 [==============================] - 869s 691ms/step - loss: 3.6290 - auc: 0.9735 - accuracy: 0.9204 - val_loss: 0.7507 - val_auc: 0.8719 - val_accuracy: 0.8011
Epoch 5/15
1256/1256 [==============================] - 833s 663ms/step - loss: 2.3360 - auc: 0.9865 - accuracy: 0.9489 - val_loss: 0.9313 - val_auc: 0.8486 - val_accuracy: 0.7954
Epoch 6/15
1256/1256 [==============================] - 833s 662ms/step - loss: 1.4005 - auc: 0.9932 - accuracy: 0.9667 - val_loss: 1.0501 - val_auc: 0.8281 - val_accuracy: 0.7914
Epoch 7/15
1256/1256 [==============================] - ETA: 0s - loss: 0.8086 - auc: 0.9970 - accuracy: 0.9818

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1/assets


1256/1256 [==============================] - 866s 689ms/step - loss: 0.8086 - auc: 0.9970 - accuracy: 0.9818 - val_loss: 0.9399 - val_auc: 0.8558 - val_accuracy: 0.8160
Epoch 8/15
1256/1256 [==============================] - 832s 662ms/step - loss: 0.5345 - auc: 0.9983 - accuracy: 0.9874 - val_loss: 1.0266 - val_auc: 0.8451 - val_accuracy: 0.8155
Epoch 9/15
1256/1256 [==============================] - 832s 662ms/step - loss: 0.3495 - auc: 0.9985 - accuracy: 0.9928 - val_loss: 1.3273 - val_auc: 0.8025 - val_accuracy: 0.7797
Epoch 10/15
1256/1256 [==============================] - 831s 662ms/step - loss: 0.2444 - auc: 0.9993 - accuracy: 0.9948 - val_loss: 1.0921 - val_auc: 0.8411 - val_accuracy: 0.8103
Epoch 11/15
1256/1256 [==============================] - 832s 662ms/step - loss: 0.1722 - auc: 0.9992 - accuracy: 0.9961 - val_loss: 1.1849 - val_auc: 0.8300 - val_accuracy: 0.8160
Epoch 12/15
1256/1256 [==============================] - 833s 663ms/step - loss: 0.1509 - auc: 0.9996 - accur

In [14]:
!cp  -r /content/logs/checkpoints/combined-model-v1 /content/drive/MyDrive/checkpoints/combined-v1-aug